In [1]:
import csv
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
import numpy as np
from datetime import datetime
import sys

In [3]:
crash_inspect_17_18 = pd.read_pickle('/home/propel/FMCSA/MCMIS/2017_2018_results/crash_inspect_17_18.pkl')

In [5]:
# Perfrom Transformations
crash_inspect_17_18['DOT_NUMBER'].fillna(0,inplace=True)
crash_inspect_17_18['DOT_NUMBER'] = crash_inspect_17_18['DOT_NUMBER'].round(0).astype(int)

In [12]:
# Perfrom Transformations
crash_inspect_17_18 = crash_inspect_17_18.loc[:,['DOT_NUMBER','INCIDENT_DATE','VEHICLE_ID_NO','CARRIER_NAME','CARRIER_STATE','INCIDENT']]

crash_inspect_17_18['Count'] = crash_inspect_17_18.groupby(['DOT_NUMBER','VEHICLE_ID_NO'])['INCIDENT_DATE'].transform('count')

crash_inspect_17_18 = crash_inspect_17_18.drop_duplicates()

crash_inspect_17_18 =  crash_inspect_17_18.sort_values(by=['VEHICLE_ID_NO'])

In [13]:
# Remove NaN from VIN
crash_inspect_17_18 = crash_inspect_17_18[crash_inspect_17_18.VEHICLE_ID_NO.notna()]
# Remove non alphanumeric from VIN
crash_inspect_17_18 = crash_inspect_17_18[crash_inspect_17_18.VEHICLE_ID_NO.str.isalnum()]

crash_inspect_17_18.shape

(8611118, 7)

In [14]:
# Perfrom Transformations
group = crash_inspect_17_18.groupby('VEHICLE_ID_NO')

df2 = group.apply(lambda x: x['DOT_NUMBER'].unique())

df2 = df2.apply(pd.Series)

df2 = df2[df2[1].notna()]

df2.head(1)

,0,1,2,3,4,5,6,7,8,9,...,420,421,422,423,424,425,426,427,428,429
VEHICLE_ID_NO,,,,,,,,,,,,,,,,,,,,,
0,807505.0,3037959.0,3048012.0,0.0,782294.0,105790.0,188556.0,64161.0,1217090.0,687831.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
# Reset index
reset_ind = df2.reset_index()

#Filter non vehicle id number types
reset_ind = reset_ind[~reset_ind['VEHICLE_ID_NO'].str.isnumeric()]
reset_ind = reset_ind[~reset_ind['VEHICLE_ID_NO'].str.isalpha()]
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != '1FUYSWDB6XL787551']
reset_ind = reset_ind[reset_ind.VEHICLE_ID_NO != 'UNKN0WN']
reset_ind.dropna(axis=1,how='all',inplace=True)
reset_ind.shape

(1464287, 32)

In [73]:
# Load OOS Carriers
OOS_data = pd.read_csv("/home/propel/FMCSA/MCMIS/CL_OS_Coords.csv")
OOS_sub = OOS_data.loc[:,['USDOT','OOS_DATE','DBA_NAME','STATE','OOS_REASON']]
OOS_sub['OOS_DATE'] = pd.to_datetime(OOS_sub['OOS_DATE']).dt.strftime('%Y-%m-%d')
OOS_sub.head(3)

,USDOT,OOS_DATE,DBA_NAME,STATE,OOS_REASON
0,1180829,2004-10-04,C & S TRUCKING,AL,New Entrant Revoked - Refusal of Audit/No Contact
1,1189597,2004-09-14,HARLIN TRUCKING,AL,New Entrant Revoked - Refusal of Audit/No Contact
2,1194141,2004-09-14,,AL,New Entrant Revoked - Refusal of Audit/No Contact


In [156]:
# Rename to USDOT
reset_ind1 = reset_ind.iloc[:,[0,1]].rename(columns={0:"USDOT"})

# Rename for merging from
crash_inspect_17_18 = crash_inspect_17_18.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result1 = pd.merge(reset_ind1, crash_inspect_17_18,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident1 =  result1.sort_values(by=['VEHICLE_ID_NO'])

Incident1.dropna(axis=0,how='any',inplace=True)
Incident1.shape

(1340495, 7)

In [155]:
# Rename to USDOT
reset_ind2 = reset_ind.iloc[:,[0,2]].rename(columns={1:"USDOT"})

# Rename for merging from
# crash_inspect_17_18 = crash_inspect_17_18.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result2 = pd.merge(reset_ind2, crash_inspect_17_18,how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident2 =  result2.sort_values(by=['VEHICLE_ID_NO'])

Incident2.dropna(axis=0,how='any',inplace=True)
Incident2.shape

(1119681, 7)

In [162]:
# Rename to USDOT
reset_ind3 = reset_ind.iloc[:,[0,3]].rename(columns={2:"USDOT"})

# # Rename for merging from
# crash_inspect_17_18 = crash_inspect_17_18.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result3 = pd.merge(reset_ind3, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident3 =  result3.sort_values(by=['VEHICLE_ID_NO'])

Incident3.dropna(axis=0,how='any',inplace=True)
Incident3.shape

(79865, 7)

In [163]:
# Rename to USDOT
reset_ind4 = reset_ind.iloc[:,[0,4]].rename(columns={3:"USDOT"})

# # Rename for merging from
# crash_inspect_17_18 = crash_inspect_17_18.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result4 = pd.merge(reset_ind4, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident4 =  result4.sort_values(by=['VEHICLE_ID_NO'])

Incident4.dropna(axis=0,how='any',inplace=True)
Incident4.shape

(6059, 7)

In [165]:
# Rename to USDOT
reset_ind5 = reset_ind.iloc[:,[0,5]].rename(columns={4:"USDOT"})

# # Rename for merging from
# crash_inspect_17_18 = crash_inspect_17_18.rename(columns={"DOT_NUMBER":"USDOT"})

# Merge on USDOT and VIN
result5 = pd.merge(reset_ind5, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident5 =  result5.sort_values(by=['VEHICLE_ID_NO'])

Incident5.dropna(axis=0,how='any',inplace=True)
Incident5.shape

(630, 7)

In [166]:
reset_ind6 = reset_ind.iloc[:,[0,6]].rename(columns={5:"USDOT"})

# Merge on USDOT and VIN
result6 = pd.merge(reset_ind6, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident6 =  result6.sort_values(by=['VEHICLE_ID_NO'])

Incident6.dropna(axis=0,how='any',inplace=True)
Incident6.shape

(106, 7)

In [167]:
reset_ind7 = reset_ind.iloc[:,[0,7]].rename(columns={6:"USDOT"})

# Merge on USDOT and VIN
result7 = pd.merge(reset_ind7, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident7 =  result7.sort_values(by=['VEHICLE_ID_NO'])

Incident7.dropna(axis=0,how='any',inplace=True)
Incident7.shape

(21, 7)

In [183]:
reset_ind8 = reset_ind.iloc[:,[0,8]].rename(columns={7:"USDOT"})

# Merge on USDOT and VIN
result8 = pd.merge(reset_ind8, crash_inspect_17_18, how='left', on=['USDOT','VEHICLE_ID_NO'])

# Sort values by 
Incident8 =  result8.sort_values(by=['VEHICLE_ID_NO'])

Incident8.dropna(axis=0,how='any',inplace=True)
Incident8.shape

(9, 7)

In [186]:
# Concat relevant Data
Incidents_17_18 = pd.concat([Incident1,Incident2,Incident3,Incident4,Incident5,Incident6,Incident7,Incident8],axis = 0)
Incidents_17_18.shape

(2546866, 7)

In [187]:
# Transform data
Incidents_17_18 =  Incidents_17_18.sort_values(by=['VEHICLE_ID_NO','INCIDENT_DATE'])
Incidents_17_18['USDOT'] = Incidents_17_18['USDOT'].round(0).astype(int)
# Incidents_17_18.reset_index(inplace=True)
Incidents_17_18.drop(['index'],axis=1,inplace=True)

In [ ]:
"""" Assumption - companies that dont share cars can't be chameleon,
     thus only the following dataframes are of interest """

In [397]:
# Vehilces with multiple USDOTs/multiple incidents
Mult_USDOT_17_18 = Incidents_17_18.groupby('VEHICLE_ID_NO').filter(lambda g: len(g) > 1).drop_duplicates(subset=['VEHICLE_ID_NO','USDOT'], keep="first")
# Remove Missing USDOTs(0)
Mult_USDOT_17_18 = Mult_USDOT_17_18[Mult_USDOT_17_18.USDOT != 0]

In [398]:
# Mult_USDOT_counts = Mult_USDOT_17_18.groupby('VEHICLE_ID_NO').USDOT.transform(lambda x : x.duplicated(keep=False).sum())

##### Engineered Features ######- Number of Carriers Per Vehicle
Mult_USDOT_17_18['No_Carriers_Per_Vehicle'] = Mult_USDOT_17_18.VEHICLE_ID_NO.map(Mult_USDOT_17_18.VEHICLE_ID_NO.value_counts())
Mult_USDOT_17_18['No_Carriers_Per_Vehicle'].shape

(711833,)

In [399]:
# Concat with OOS trucking companies
MultUSDOT_OOS = Mult_USDOT_17_18.reset_index().merge(OOS_sub, how="left",on= ['USDOT']).set_index('index')

,VEHICLE_ID_NO,USDOT,INCIDENT_DATE,CARRIER_NAME,CARRIER_STATE,INCIDENT,Count,No_Carriers_Per_Vehicle,OOS_DATE,DBA_NAME,STATE,OOS_REASON
index,,,,,,,,,,,,
1,00000000000EB2677,606698,2017-01-20,TRANSPORTES JD,CI,INSPECTION,2.0,1,NaN,NaN,NaN,NaN
3,0000000000105103N,2351774,2017-02-06,WERNER & SONS WELL DRILLING,PA,INSPECTION,1.0,1,NaN,NaN,NaN,NaN
6,0000000000166006S,2923598,2018-03-22,ONE CALL CONTAINER LLC,PA,INSPECTION,1.0,1,NaN,NaN,NaN,NaN
9,0000000000T886756,2830357,2018-04-12,ARIEL RODRIGUEZ SOTO,FL,INSPECTION,2.0,1,2019-12-16,,FL,New Entrant Revoked - Refusal of Audit/No Contact
13,000000000CA498632,2963250,2018-01-18,ROBERTO CASTILLO,CA,INSPECTION,1.0,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2546843,Z9AD4021WA0126609,1507993,2018-02-09,BROADWAY WOOD CORP,LA,INSPECTION,2.0,1,NaN,NaN,NaN,NaN
2546856,ZP11460,900563,2018-05-02,GABRIEL V PAMPLONA,CA,INSPECTION,1.0,1,NaN,NaN,NaN,NaN
2546859,ZP51945,636861,2018-04-30,BEST TRANSPORTATION LLC,AZ,INSPECTION,3.0,1,NaN,NaN,NaN,NaN


In [400]:
# Drop Na from above- OOS Carriers Only
Mult_OOS_Na = MultUSDOT_OOS.dropna(subset=['OOS_DATE'])
Mult_OOS_Na = Mult_OOS_Na.sort_values(by=['VEHICLE_ID_NO'])
Mult_OOS_Na

,VEHICLE_ID_NO,USDOT,INCIDENT_DATE,CARRIER_NAME,CARRIER_STATE,INCIDENT,Count,No_Carriers_Per_Vehicle,OOS_DATE,DBA_NAME,STATE,OOS_REASON
index,,,,,,,,,,,,
9,0000000000T886756,2830357,2018-04-12,ARIEL RODRIGUEZ SOTO,FL,INSPECTION,2.0,1,2019-12-16,,FL,New Entrant Revoked - Refusal of Audit/No Contact
1052,109TT4835MB409010,3052345,2018-04-02,AMERICAN ALLIED ALABAMA LLC,AL,INSPECTION,1.0,2,2018-05-21,A&A AUTOMOTIVE LLC,AL,New Entrant Revoked - Refusal of Audit/No Contact
1132,10BAB62U41F0B3300,2481333,2018-04-19,RICARDO CHAVEZ,TX,INSPECTION,1.0,2,2018-06-04,OUTLIER TRANSPORTS,TX,New Entrant Revoked - Refusal of Audit/No Contact
5466,10ZL42205U1001091,2524616,2017-01-31,DERON ALSTON CLAYTON,NC,INSPECTION,1.0,4,2017-08-28,,NC,New Entrant Revoked - Refusal of Audit/No Contact
6049,119AS483018540128,3069838,2018-03-21,MAKING MOVES TRANSPORT,FL,INSPECTION,3.0,1,2019-01-30,,FL,New Entrant Revoked - Failure of Safety Audit
...,...,...,...,...,...,...,...,...,...,...,...,...
2545557,YE2TC62B2X2043782,2595362,2018-01-05,AYLARD TOURS LLC,TX,INSPECTION,2.0,1,2019-07-05,,TX,Unsatisfactory = Unfit
2545632,YE2TC63B3X2043420,2595362,2018-02-09,AYLARD TOURS LLC,TX,INSPECTION,2.0,1,2019-07-05,,TX,Unsatisfactory = Unfit
2545656,YE2TC73B6P2024592,2488995,2017-01-13,RAY GUTIERREZ,CA,INSPECTION,1.0,1,2016-07-25,R G TRIPS,CA,Unsatisfactory = Unfit


In [497]:
# Format Date
MultUSDOT_OOS['OOS_DATE'] = pd.to_datetime(MultUSDOT_OOS['OOS_DATE'], format='%Y-%m-%d')
MultUSDOT_OOS['OOS_DATE'] = MultUSDOT_OOS['OOS_DATE'].dt.date

In [555]:
# This file has OOS and In service sharing a vehicle
OOS_both = MultUSDOT_OOS[MultUSDOT_OOS.set_index(['VEHICLE_ID_NO']).index.isin(Mult_OOS_Na.set_index(['VEHICLE_ID_NO']).index)]
# OOS_both = OOS_both.sort_values(by=['VEHICLE_ID_NO'])
OOS_both.head(10)

,VEHICLE_ID_NO,USDOT,INCIDENT_DATE,CARRIER_NAME,CARRIER_STATE,INCIDENT,Count,No_Carriers_Per_Vehicle,OOS_DATE,DBA_NAME,STATE,OOS_REASON
index,,,,,,,,,,,,
9,0000000000T886756,2830357,2018-04-12,ARIEL RODRIGUEZ SOTO,FL,INSPECTION,2.0,1,2019-12-16,,FL,New Entrant Revoked - Refusal of Audit/No Contact
1052,109TT4835MB409010,3052345,2018-04-02,AMERICAN ALLIED ALABAMA LLC,AL,INSPECTION,1.0,2,2018-05-21,A&A AUTOMOTIVE LLC,AL,New Entrant Revoked - Refusal of Audit/No Contact
1053,109TT4835MB409010,2968068,2018-05-14,L E B UNLIMITED LLC,FL,INSPECTION,1.0,2,NaT,NaN,NaN,NaN
1131,10BAB62U41F0B3300,2195271,2018-04-17,XROSS INTEGRATED TRANSPORT,TX,INSPECTION,1.0,2,NaT,NaN,NaN,NaN
1132,10BAB62U41F0B3300,2481333,2018-04-19,RICARDO CHAVEZ,TX,INSPECTION,1.0,2,2018-06-04,OUTLIER TRANSPORTS,TX,New Entrant Revoked - Refusal of Audit/No Contact
5466,10ZL42205U1001091,2524616,2017-01-31,DERON ALSTON CLAYTON,NC,INSPECTION,1.0,4,2017-08-28,,NC,New Entrant Revoked - Refusal of Audit/No Contact
5467,10ZL42205U1001091,3062952,2018-01-30,3KKK TRUCKING,NC,INSPECTION,3.0,4,NaT,NaN,NaN,NaN
5468,10ZL42205U1001091,3096305,2018-02-27,ALVIN AUSTIN TRUCKING,NC,INSPECTION,1.0,4,NaT,NaN,NaN,NaN
5471,10ZL42205U1001091,3074421,2018-06-11,3-K TRUCKING,NC,INSPECTION,1.0,4,NaT,NaN,NaN,NaN


In [582]:
# Difference in days between IS Carriers and OOS Carrier Incident Date
Days_Between = OOS_both.sort_values(['VEHICLE_ID_NO','INCIDENT_DATE']).groupby('VEHICLE_ID_NO')['INCIDENT_DATE'].transform(lambda x: x.min() - x)
Days_Between.astype('timedelta64[D]')
Days_Between = Days_Between/pd.Timedelta(1, unit='d')
Days_Between.head(15)

index
9         0.0
1052      0.0
1053    -42.0
1131      0.0
1132     -2.0
5466      0.0
5467   -364.0
5468   -392.0
5471   -496.0
6049      0.0
6428      0.0
6429   -435.0
6691      0.0
6804      0.0
6865      0.0
Name: INCIDENT_DATE, dtype: float64

In [604]:
# Groupby VIN and subtract dates
cols = ['INCIDENT_DATE','OOS_DATE']
OOS_boths = OOS_both.sort_values(['VEHICLE_ID_NO','INCIDENT_DATE']).groupby('VEHICLE_ID_NO')[cols].diff(-1).fillna(0)

In [605]:
OOS_boths.rename(columns={'INCIDENT_DATE':'Days_Between_Inc'},inplace=True)
# Remove Seconds etc.
OOS_boths['Days_Between_Inc'] = OOS_boths['Days_Between_Inc'].astype('timedelta64[D]')
OOS_boths['Days_Between_Inc'] = OOS_boths['Days_Between_Inc']/pd.Timedelta(1, unit='d')

OOS_boths.head(10)

,Days_Between_Inc,OOS_DATE
index,,
9,0.0,0
1052,-42.0,0
1053,0.0,0
1131,-2.0,0
1132,0.0,0
5466,-364.0,0
5467,-28.0,0
5468,-104.0,0
5471,0.0,0


In [620]:
OOS_boths = OOS_boths.join(Days_Between, how='inner',lsuffix='_left', rsuffix='_right')
OOS_boths["Days_Between_OOSIncident_ISIncident"] = OOS_boths[["Days_Between_Inc", "INCIDENT_DATE_left"]].min(axis=1)
OOS_boths_dt = OOS_boths["Days_Between_OOSIncident_ISIncident"].abs()

In [621]:
##### New Feature ########### - Days_Between_ShutDown an OOS Incident and an In Service Carrier Incident
# Replace all zeros with arbitrary large number for clustering
OOS_boths_dt = OOS_boths_dt.replace(0, 1000000)
OOS_boths_dt.head(20)

index
9       1000000.0
1052         42.0
1053         42.0
1131          2.0
1132          2.0
5466        364.0
5467        364.0
5468        392.0
5471        496.0
6049    1000000.0
6428        435.0
6429        435.0
6691    1000000.0
6804    1000000.0
6865          7.0
6866          7.0
7140         49.0
7141         49.0
7246        380.0
7248        380.0
Name: Days_Between_OOSIncident_ISIncident, dtype: float64

In [473]:
OOS_3nd = OOS_both.sort_values(['VEHICLE_ID_NO','INCIDENT_DATE']).groupby('VEHICLE_ID_NO')[cols].diff(3).fillna(0)
OOS_3nd.rename(columns={'INCIDENT_DATE':'Days_Between_ShutDown1_Incident2'},inplace=True)

# Remove Time
OOS_3nd['Days_Between_ShutDown1_Incident2'] = OOS_3nd['Days_Between_ShutDown1_Incident2'].astype('timedelta64[D]')
OOS_3nd['Days_Between_ShutDown1_Incident2'] = OOS_3nd['Days_Between_ShutDown1_Incident2']/ pd.Timedelta(1, unit='d')
# Abs Value
OOS_3nd['Days_Between_ShutDown1_Incident2'] = OOS_3nd['Days_Between_ShutDown1_Incident2'].abs()
OOS_3nd.head(10)


,Days_Between_ShutDown1_Incident2,OOS_DATE
index,,
9,0.0,0
1052,0.0,0
1053,0.0,0
1131,0.0,0
1132,0.0,0
5466,0.0,0
5467,0.0,0
5468,0.0,0
5471,496.0,0


In [456]:
# Incident date after oos date
Indt_after_17_18 = OOS_both.loc[OOS_both['OOS_DATE'] < OOS_both['INCIDENT_DATE']]


### New Feature #### - Incident date after oos date
Indt_after_17_18 = Indt_after_17_18.assign(Incident_After_OOS = 1)

# Indt_after_17_18 =  Indt_after_17_18.sort_values(by=['VEHICLE_ID_NO'])

In [457]:
Indt_after_17_18.head(10)

,VEHICLE_ID_NO,USDOT,INCIDENT_DATE,CARRIER_NAME,CARRIER_STATE,INCIDENT,Count,No_Carriers_Per_Vehicle,OOS_DATE,DBA_NAME,STATE,OOS_REASON,Incident_After_OOS
index,,,,,,,,,,,,,
21480,13N24830X25913096,2504063,2018-01-04,TAINO TRUCKING LLC,LA,INSPECTION,1.0,1,2015-01-06,,LA,New Entrant Revoked - Failure of Safety Audit,1
32365,16VGX4023F6094341,2369488,2018-01-12,MG LOGISTICS WAREHOUSE & FREIGHT SERVICES,TX,INSPECTION,1.0,2,2013-12-17,,TX,New Entrant Revoked - Failure of Safety Audit,1
41163,1B9A13135LA180590,2504063,2018-02-27,TAINO TRUCKING LLC,LA,INSPECTION,2.0,1,2015-01-06,,LA,New Entrant Revoked - Failure of Safety Audit,1
42595,1B9CS4529YP275174,2102244,2017-01-03,ELOHE TRANSPORT,TX,INSPECTION,2.0,2,2016-11-28,AUTOS ELOHE,TX,Unsatisfactory = Unfit,1
44766,1B9PA4225VFBJ1353,2341701,2017-01-09,JK HANSLEY TRUCKING LLC,NC,INSPECTION,1.0,2,2015-11-18,,NC,New Entrant Revoked - Failure of Safety Audit,1
46116,1BABHCKA98F251577,2282376,2018-03-12,JOSUE GARCIA,TX,INSPECTION,1.0,1,2013-09-03,INGARE AUTO REPUESTOS,TX,New Entrant Revoked - Refusal of Audit/No Contact,1
49158,1BAKGCPH2FF307672,2087821,2017-09-27,ADVANCED CAR SERVICE,NJ,CRASH,1.0,2,2014-04-29,,NJ,New Entrant Revoked - Refusal of Audit/No Contact,1
53279,1C91140216S770595,2058806,2018-03-21,JOSE GARCIA,TX,INSPECTION,1.0,2,2017-11-20,,TX,New Entrant Revoked - Refusal of Audit/No Contact,1
55938,1C9AV1641WG148609,2102244,2017-02-24,ELOHE TRANSPORT,TX,INSPECTION,11.0,1,2016-11-28,AUTOS ELOHE,TX,Unsatisfactory = Unfit,1


In [ ]:
 """ Append New Features to Dataframe """

In [640]:
# Incident date after oos date
Incidents_after = Indt_after_17_18['Incident_After_OOS']

# Days_Between_ShutDown an OOS Incident and an In Service Carrier Incident
OOS_boths_dt

# Number of Carriers Per Vehicle
#MultUSDOT_OOS['No_Carriers_Per_Vehicle']


# Join on indicies
Feat_17_18 = MultUSDOT_OOS.join(OOS_boths_dt, how='outer')

Eng_Features_17_18 = Feat_17_18.join(Incidents_after, how='outer')


# Replace Nans for Clustering
Eng_Features_17_18['Days_Between_OOSIncident_ISIncident'] = Eng_Features_17_18['Days_Between_OOSIncident_ISIncident'].replace(np.nan, 1000000)
# Replace Nans for Clustering
Eng_Features_17_18['Incident_After_OOS'] = Eng_Features_17_18['Incident_After_OOS'].replace(np.nan,0)

In [653]:
Eng_Features_17_18[Eng_Features_17_18['Days_Between_OOSIncident_ISIncident'] < 20].head(20)

,VEHICLE_ID_NO,USDOT,CARRIER_NAME,CARRIER_STATE,No_Carriers_Per_Vehicle,Days_Between_OOSIncident_ISIncident,Incident_After_OOS
index,,,,,,,
1131,10BAB62U41F0B3300,2195271,XROSS INTEGRATED TRANSPORT,TX,2,2.0,0.0
1132,10BAB62U41F0B3300,2481333,RICARDO CHAVEZ,TX,2,2.0,0.0
6865,13N145208S1565723,2254108,HURBO TRANSPORTS LLC,TX,2,7.0,0.0
6866,13N145208S1565723,2023717,JOSE LUIS HURTADO BORBOLLA,CH,2,7.0,0.0
11748,13N1482C0H1520702,1357847,D LAMONTANA TRANSPORT INC,IL,2,3.0,0.0
11749,13N1482C0H1520702,2335691,D AND L STEEL TRANSPORT LLC,IL,2,3.0,0.0
33156,16VGX5120G2039942,2839749,SCEIFRESTRANSPORTATION LLC,AR,2,1.0,0.0
33157,16VGX5120G2039942,336459,LOWERY CARNIVAL CO INC,LA,2,1.0,0.0
44766,1B9PA4225VFBJ1353,2341701,JK HANSLEY TRUCKING LLC,NC,2,9.0,1.0


In [642]:
# Increase CSV size
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [643]:
#Save to CSV

Eng_Features_17_18 = Eng_Features_17_18.loc[:,['VEHICLE_ID_NO','USDOT','CARRIER_NAME','CARRIER_STATE','No_Carriers_Per_Vehicle','Days_Between_OOSIncident_ISIncident','Incident_After_OOS']]
Eng_Features_17_18.to_csv('/home/propel/FMCSA/MCMIS/2017_2018_results/Eng_Features_17_18.csv',index = False)